In [2]:

from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Practice').getOrCreate()

24/08/19 09:38:04 WARN Utils: Your hostname, Shelly-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.115 instead (on interface en0)
24/08/19 09:38:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/19 09:38:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/19 09:38:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark

24/08/19 09:38:17 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [27]:
df_pyspark=spark.read.csv('tt2.csv', header=True, inferSchema=True)

In [7]:
df_pyspark.show()

+----+----+----+------+
|Name| Age| Exp|Salary|
+----+----+----+------+
|   A|  11|   1|   100|
|   B|  23|   2|   200|
|   C|  45|   3|   300|
|   D|  12|   1|   400|
|   E|   3|   2|   500|
|   F|   4|   3|  NULL|
|   G|   5|   4|   500|
|NULL|   6|   5|   600|
|   I|NULL|NULL|   100|
|NULL|   4|NULL|  NULL|
+----+----+----+------+



In [9]:
#drop the column
df_pyspark.drop('Name').show()

+----+----+------+
| Age| Exp|Salary|
+----+----+------+
|  11|   1|   100|
|  23|   2|   200|
|  45|   3|   300|
|  12|   1|   400|
|   3|   2|   500|
|   4|   3|  NULL|
|   5|   4|   500|
|   6|   5|   600|
|NULL|NULL|   100|
|   4|NULL|  NULL|
+----+----+------+



In [10]:
#drop na
df_pyspark.na.drop().show()

+----+---+---+------+
|Name|Age|Exp|Salary|
+----+---+---+------+
|   A| 11|  1|   100|
|   B| 23|  2|   200|
|   C| 45|  3|   300|
|   D| 12|  1|   400|
|   E|  3|  2|   500|
|   G|  5|  4|   500|
+----+---+---+------+



In [15]:
#any=all, only drop when all values are null
df_pyspark.na.drop(how="all").show()
df_pyspark.na.drop(how="any").show()

#thresh=2, at least 2 non-null values should be present
df_pyspark.na.drop(how="any", thresh=2).show()

+----+----+----+------+
|Name| Age| Exp|Salary|
+----+----+----+------+
|   A|  11|   1|   100|
|   B|  23|   2|   200|
|   C|  45|   3|   300|
|   D|  12|   1|   400|
|   E|   3|   2|   500|
|   F|   4|   3|  NULL|
|   G|   5|   4|   500|
|NULL|   6|   5|   600|
|   I|NULL|NULL|   100|
|NULL|   4|NULL|  NULL|
+----+----+----+------+

+----+---+---+------+
|Name|Age|Exp|Salary|
+----+---+---+------+
|   A| 11|  1|   100|
|   B| 23|  2|   200|
|   C| 45|  3|   300|
|   D| 12|  1|   400|
|   E|  3|  2|   500|
|   G|  5|  4|   500|
+----+---+---+------+

+----+----+----+------+
|Name| Age| Exp|Salary|
+----+----+----+------+
|   A|  11|   1|   100|
|   B|  23|   2|   200|
|   C|  45|   3|   300|
|   D|  12|   1|   400|
|   E|   3|   2|   500|
|   F|   4|   3|  NULL|
|   G|   5|   4|   500|
|NULL|   6|   5|   600|
|   I|NULL|NULL|   100|
+----+----+----+------+



In [16]:
#subset, drop na values only for specify columns
df_pyspark.na.drop(how="any", subset=["Age"]).show()

+----+---+----+------+
|Name|Age| Exp|Salary|
+----+---+----+------+
|   A| 11|   1|   100|
|   B| 23|   2|   200|
|   C| 45|   3|   300|
|   D| 12|   1|   400|
|   E|  3|   2|   500|
|   F|  4|   3|  NULL|
|   G|  5|   4|   500|
|NULL|  6|   5|   600|
|NULL|  4|NULL|  NULL|
+----+---+----+------+



In [23]:
## Filling the missing values
df_pyspark.na.fill('Missing Values', ['Name', 'Age']).show()





+--------------+--------------+----+------+
|          Name|           Age| Exp|Salary|
+--------------+--------------+----+------+
|             A|            11|   1|   100|
|             B|            23|   2|   200|
|             C|            45|   3|   300|
|             D|            12|   1|   400|
|             E|             3|   2|   500|
|             F|             4|   3|  NULL|
|             G|             5|   4|   500|
|Missing Values|             6|   5|   600|
|             I|Missing Values|NULL|   100|
|Missing Values|             4|NULL|  NULL|
+--------------+--------------+----+------+



In [37]:
## Filling with mean values
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=["Age", "Exp", "Salary"],
    outputCols=["{}_imputed".format(c) for c in ["Age", "Exp", "Salary"]]
)

In [38]:
imputer.setStrategy("mean").fit(df_pyspark).transform(df_pyspark).show()

+----+----+----+------+-----------+-----------+--------------+
|Name| Age| Exp|Salary|Age_imputed|Exp_imputed|Salary_imputed|
+----+----+----+------+-----------+-----------+--------------+
|   A|  11|   1|   100|         11|          1|           100|
|   B|  23|   2|   200|         23|          2|           200|
|   C|  45|   3|   300|         45|          3|           300|
|   D|  12|   1|   400|         12|          1|           400|
|   E|   3|   2|   500|          3|          2|           500|
|   F|   4|   3|  NULL|          4|          3|           337|
|   G|   5|   4|   500|          5|          4|           500|
|NULL|   6|   5|   600|          6|          5|           600|
|   I|NULL|NULL|   100|         12|          2|           100|
|NULL|   4|NULL|  NULL|          4|          2|           337|
+----+----+----+------+-----------+-----------+--------------+



24/08/19 12:06:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1112271 ms exceeds timeout 120000 ms
24/08/19 12:06:49 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/19 12:06:50 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$